In [2]:
import torch
import torch.nn.functional as F
import numpy as np

In [3]:
b_s = 2
candidate_logprob = torch.rand((5,3,2))

In [60]:
candidate_logprob = torch.rand((2, 1, 500))
candlog = candidate_logprob.view(b_s, -1)
# print(candlog, torch.max(candlog))
probs = F.softmax(candlog, dim=-1)
# print(probs)
temp = .2
if temp is not None:
    samp_probs = F.softmax(candlog.div_(temp), dim=-1)
else:
    samp_probs = probs.clone()
# print("after temp:",samp_probs)
k = 5
m = None
if k is not None:
            print(torch.max(samp_probs[0]), torch.min(samp_probs))
#             print(torch.topk(samp_probs, k)[0][..., -1, None])

            print("top k", torch.topk(samp_probs, k)[0][..., -1, None])
            indices_to_remove = samp_probs < torch.topk(samp_probs, k)[0][..., -1, None]
            tot = torch.sum(~indices_to_remove)
            print(tot)
            samp_probs[indices_to_remove] = 0
            for val in samp_probs[0]:
                if val>0:
                    print("val is", val)
            if m is not None:
                samp_probs.div_(samp_probs.sum(1).unsqueeze(1))
                samp_probs.mul_(1-m)
                samp_probs.add_(probs.mul(m))
#             next_tokens = torch.stack([samp_probs_batch.multinomial(5,replacement=False) for samp_probs_batch in samp_probs], 0)
            next_tokens = samp_probs.multinomial(5,replacement=False) 
            print("next",next_tokens, next_tokens.view(-1, 1).size(), samp_probs.size())
            next_logprobs = samp_probs.gather(1, next_tokens)
            print(next_logprobs.size())
p = 0.6 
sorted_probs, sorted_indices = torch.sort(samp_probs, descending=True)
cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
sorted_indices_to_remove = cumulative_probs > p
sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
sorted_indices_to_remove[:, 0] = 0
sorted_samp_probs = sorted_probs.clone()
sorted_samp_probs[sorted_indices_to_remove] = 0
# if m is not None:
#     sorted_samp_probs.div_(sorted_samp_probs.sum(1).unsqueeze(1))
#     sorted_samp_probs.mul_(1-m)
#     sorted_samp_probs.add_(sorted_probs.mul(m))
sorted_next_indices = sorted_samp_probs.multinomial(1).view(-1, 1)
next_tokens = sorted_indices.gather(1, sorted_next_indices)
next_logprobs = sorted_samp_probs.gather(1, sorted_next_indices).log()
# print(samp_probs, next_tokens)

tensor(0.0101) tensor(6.6603e-05)
top k tensor([[0.0099],
        [0.0091]])
tensor(10)
val is tensor(0.0101)
val is tensor(0.0101)
val is tensor(0.0100)
val is tensor(0.0100)
val is tensor(0.0099)
next tensor([[390, 192, 185, 402,  53],
        [264, 183, 109, 404, 203]]) torch.Size([10, 1]) torch.Size([2, 500])
torch.Size([2, 5])


In [27]:
a = torch.rand((4,5,6))
a.view(-1,1).size()


torch.Size([120, 1])

In [52]:
sorted_samp_probs

tensor([[0.8077, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3779, 0.3530, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5433, 0.3198, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3612, 0.2891, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3201, 0.2886, 0.0000, 0.0000, 0.0000, 0.0000]])

In [46]:
next_logprobs

tensor([[0.6494],
        [0.5888],
        [0.3720],
        [0.3226],
        [0.4562]])

In [12]:
selected_logprob, selected_idx = torch.sort(candidate_logprob.view(b_s, -1), -1, descending=True)

In [13]:
print(selected_logprob, selected_idx)

tensor([[0.6883, 0.6707, 0.5767, 0.4844, 0.4069, 0.0211],
        [0.6070, 0.5587, 0.5398, 0.3479, 0.3192, 0.1867],
        [0.7811, 0.7079, 0.5025, 0.2493, 0.2076, 0.1847],
        [0.9034, 0.7155, 0.4303, 0.3348, 0.1552, 0.0152],
        [0.7834, 0.7049, 0.4613, 0.4049, 0.2520, 0.1491]]) tensor([[3, 2, 5, 0, 4, 1],
        [5, 3, 2, 0, 1, 4],
        [5, 2, 3, 0, 4, 1],
        [2, 5, 3, 0, 1, 4],
        [5, 3, 0, 1, 4, 2]])


In [14]:
beam_size = 2

In [15]:
selected_logprob, selected_idx = selected_logprob[:, :beam_size], selected_idx[:, :beam_size]

In [16]:
print(selected_logprob, selected_idx)

tensor([[0.6883, 0.6707],
        [0.6070, 0.5587],
        [0.7811, 0.7079],
        [0.9034, 0.7155],
        [0.7834, 0.7049]]) tensor([[3, 2],
        [5, 3],
        [5, 2],
        [2, 5],
        [5, 3]])


In [64]:
a = torch.rand((3,4))
sorteda, indx = a.sort()
print(a)
print(indx)
b = a[indx]
print(a,b)

tensor([[0.1514, 0.3058, 0.5706, 0.1889],
        [0.9444, 0.0556, 0.0396, 0.5188],
        [0.9170, 0.8527, 0.2875, 0.9786]])
tensor([[0, 3, 1, 2],
        [2, 1, 3, 0],
        [2, 1, 0, 3]])


IndexError: index 3 is out of bounds for dimension 0 with size 3